In [ ]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 749.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 r

In [ ]:
pip install gensim==4.3.3 numpy==1.24.3 scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python

### Loading the Word2Vec Model

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec.load('/content/youtube_word2vec_200dim.model')
print("Word2Vec model loaded successfully!")

Word2Vec model loaded successfully!


### Loading the Tokenized excel file

In [ ]:
import pandas as pd
import ast
# Load dataset
df = pd.read_excel("/content/sentiment_tokens.xlsx")

# Safe parsing: Convert stringified list to actual list
def safe_parse(x):
    try:
        return ast.literal_eval(x)
    except Exception as e:
        print(f"Error parsing row: {x[:80]}... -> {e}")
        return []

df['tokens'] = df['tokens'].apply(safe_parse)



Error parsing row: ['sidney', 'powel', 'credible', 'rolling_on_the_floor_laughing', 'rolling_on_the... -> '[' was never closed (<unknown>, line 1)
Error parsing row: ['relateable', 'party_popper', 'party_popper', 'party_popper', 'party_popper', '... -> '[' was never closed (<unknown>, line 1)


In [ ]:
df.shape

(963055, 2)

In [ ]:
print(df['tokens'].iloc[0])
print(type(df['tokens'].iloc[0]))  # Should now be <class 'list'>

['anyone', 'know', 'movie', '?']
<class 'list'>


### Create Sentence Embeddings (Mean Pooling)

In [ ]:
# BLOCK 3: Create Sentence Embeddings (Mean Pooling)

import numpy as np
from tqdm import tqdm

def sentence_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Build feature matrix X
X = np.array([sentence_embedding(tokens, model.wv) for tokens in tqdm(df['tokens'])])


100%|██████████| 963055/963055 [00:46<00:00, 20884.72it/s]


In [ ]:
X

array([[-0.140857  , -0.03284824, -0.06331647, ..., -0.17949659,
        -0.12216   , -0.14116634],
       [ 0.04501991,  0.01944968,  0.2220042 , ..., -0.11654779,
         0.0016356 ,  0.05193744],
       [ 0.07193813,  0.07645284, -0.25776291, ..., -0.05467423,
         0.0794609 , -0.17267951],
       ...,
       [-0.0154625 ,  0.11621616,  0.0079456 , ..., -0.12408878,
         0.07909657,  0.13012357],
       [ 0.02637978,  0.01195572,  0.09002316, ..., -0.19372623,
         0.01496309,  0.01822849],
       [-0.17009021,  0.01112945, -0.27889833, ..., -0.32648149,
         0.15604128,  0.17522995]])

In [ ]:
df.columns

Index(['Sentiment', 'tokens'], dtype='object')

In [ ]:
y = df['Sentiment'].values

In [ ]:
y

array(['Neutral', 'Positive', 'Neutral', ..., 'Negative', 'Neutral',
       'Neutral'], dtype=object)

In [ ]:
# BLOCK 4: Encode Labels & Train-Test Split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Classes: {label_encoder.classes_}")


Classes: ['Negative' 'Neutral' 'Positive']


In [ ]:
y_encoded

array([1, 2, 1, ..., 0, 1, 1])

### Traning the Sentence Embedding created on ML models

#### Deafult Logistic Regression Accuracy

In [ ]:
#  BLOCK 5: Train Models & Evaluate


In [ ]:
#  Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log))
print("Classification Report:\n", classification_report(y_test, y_pred_log, target_names=label_encoder.classes_))


=== Logistic Regression ===
Accuracy: 0.6603361178748878
Confusion Matrix:
 [[45173 14055  5960]
 [15940 36136 10055]
 [ 9113 10300 45879]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.64      0.69      0.67     65188
     Neutral       0.60      0.58      0.59     62131
    Positive       0.74      0.70      0.72     65292

    accuracy                           0.66    192611
   macro avg       0.66      0.66      0.66    192611
weighted avg       0.66      0.66      0.66    192611



#### Default Random Forest Accuracy

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("=== Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf, target_names=label_encoder.classes_))


KeyboardInterrupt: 

#### Default Xgboost Accuracy

In [ ]:
# XGBoostXG
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax', num_class=len(label_encoder.classes_),
    eval_metric='mlogloss', use_label_encoder=False
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("=== XGBoost ===")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
print("Classification Report:\n", classification_report(y_test, y_pred_xgb, target_names=label_encoder.classes_))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:06:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


=== XGBoost ===
Accuracy: 0.6852256620857583
Confusion Matrix:
 [[47566 12287  5335]
 [15458 37879  8794]
 [ 9348  9407 46537]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.66      0.73      0.69     65188
     Neutral       0.64      0.61      0.62     62131
    Positive       0.77      0.71      0.74     65292

    accuracy                           0.69    192611
   macro avg       0.69      0.68      0.68    192611
weighted avg       0.69      0.69      0.69    192611



#### Default LightGBM Accuracy

In [ ]:
# LightGBM
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=len(label_encoder.classes_))
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

print("=== LightGBM ===")
print("Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lgb))
print("Classification Report:\n", classification_report(y_test, y_pred_lgb, target_names=label_encoder.classes_))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.236304 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 770444, number of used features: 200
[LightGBM] [Info] Start training from score -1.083397
[LightGBM] [Info] Start training from score -1.131428
[LightGBM] [Info] Start training from score -1.081803


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


=== LightGBM ===
Accuracy: 0.6715296634148621
Confusion Matrix:
 [[46826 13101  5261]
 [16248 37293  8590]
 [10045 10022 45225]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.64      0.72      0.68     65188
     Neutral       0.62      0.60      0.61     62131
    Positive       0.77      0.69      0.73     65292

    accuracy                           0.67    192611
   macro avg       0.67      0.67      0.67    192611
weighted avg       0.68      0.67      0.67    192611



In [ ]:
# === XGBoost (GPU) ===
print("\n=== XGBoost (GPU) ===")
xgb_model = xgb.XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    max_depth=10,
    n_estimators=100,
    random_state=42,
    verbosity=1,
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
print("Classification Report:\n", classification_report(y_test, y_pred_xgb, target_names=label_encoder.classes_))


=== XGBoost (GPU) ===


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:55] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:22:55] WARNING: /workspace/src/common/error_ms

Accuracy: 0.6961025071257613
Confusion Matrix:
 [[48031 12098  5059]
 [14569 39126  8436]
 [ 8786  9586 46920]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.67      0.74      0.70     65188
     Neutral       0.64      0.63      0.64     62131
    Positive       0.78      0.72      0.75     65292

    accuracy                           0.70    192611
   macro avg       0.70      0.70      0.70    192611
weighted avg       0.70      0.70      0.70    192611



In [ ]:
# === LightGBM (GPU optional) ===
print("\n=== LightGBM ===")
lgbm_model = lgb.LGBMClassifier(
    device='gpu',     # Remove this line if using CPU
    max_depth=10,
    num_leaves=31,
    n_estimators=100,
    random_state=42
)
lgbm_model.fit(X_train, y_train)
y_pred_lgb = lgbm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lgb))
print("Classification Report:\n", classification_report(y_test, y_pred_lgb, target_names=label_encoder.classes_))


=== LightGBM ===


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 770444, number of used features: 200
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 200 dense feature groups (146.95 MB) transferred to GPU in 0.207184 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -1.083397
[LightGBM] [Info] Start training from score -1.131428
[LightGBM] [Info] Start training from score -1.081803


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Accuracy: 0.6713323745788143
Confusion Matrix:
 [[46750 13218  5220]
 [16224 37301  8606]
 [ 9993 10044 45255]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.64      0.72      0.68     65188
     Neutral       0.62      0.60      0.61     62131
    Positive       0.77      0.69      0.73     65292

    accuracy                           0.67    192611
   macro avg       0.67      0.67      0.67    192611
weighted avg       0.68      0.67      0.67    192611



In [ ]:
# === Logistic Regression ===
print("\n=== Logistic Regression ===")
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))
print("Classification Report:\n", classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))


=== Logistic Regression ===
Accuracy: 0.6603361178748878
Confusion Matrix:
 [[45173 14055  5960]
 [15940 36136 10055]
 [ 9113 10300 45879]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.64      0.69      0.67     65188
     Neutral       0.60      0.58      0.59     62131
    Positive       0.74      0.70      0.72     65292

    accuracy                           0.66    192611
   macro avg       0.66      0.66      0.66    192611
weighted avg       0.66      0.66      0.66    192611



### LightGBM and Xgboost for Deeper Hyper-parameter Tuning

### Hyper-parameter tuning via Optuna

In [ ]:
pip install optuna lightgbm xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.2 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y, random_state=42)


In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((770444, 200), (192611, 200), (770444,), (192611,))

In [ ]:
y_train

array([2, 2, 1, ..., 1, 0, 0])

### LightGBM hyper-parameter tuning

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score
import joblib
import numpy as np


In [ ]:

# Make sure y, X_train, X_val, y_train, y_val are defined

# -------------------------------
# LIGHTGBM OPTUNA TUNING + SAVE
# -------------------------------
def objective_lgb(trial):
    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'num_class': len(set(y)),
        'verbosity': -1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
    }

    train_set = lgb.Dataset(X_train, label=y_train)
    val_set = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        param,
        train_set,
        valid_sets=[val_set],
        callbacks=[
            early_stopping(stopping_rounds=50),
            log_evaluation(period=10)
        ]
    )

    preds = model.predict(X_val)
    pred_labels = preds.argmax(axis=1)
    return accuracy_score(y_val, pred_labels)

# Run LightGBM tuning
study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("✅ Best LightGBM params:", study_lgb.best_params)

# Train final LightGBM model with best params
best_params_lgb = study_lgb.best_params
best_params_lgb.update({
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_class': len(set(y)),
    'verbosity': -1,
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
})

train_set = lgb.Dataset(X_train, label=y_train)
val_set = lgb.Dataset(X_val, label=y_val)

final_lgb = lgb.train(
    best_params_lgb,
    train_set,
    valid_sets=[val_set],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=10)
    ]
)

# Save final LightGBM model
joblib.dump(final_lgb, 'best_lightgbm.pkl')
print("✅ LightGBM model saved to best_lightgbm.pkl")

[I 2025-05-22 14:52:32,134] A new study created in memory with name: no-name-40e4a345-e895-4afb-8d6c-b5952470ffba


Training until validation scores don't improve for 50 rounds
[10]	valid_0's multi_logloss: 0.817268
[20]	valid_0's multi_logloss: 0.775915
[30]	valid_0's multi_logloss: 0.757608
[40]	valid_0's multi_logloss: 0.746918
[50]	valid_0's multi_logloss: 0.739437
[60]	valid_0's multi_logloss: 0.733955
[70]	valid_0's multi_logloss: 0.729262
[80]	valid_0's multi_logloss: 0.725492
[90]	valid_0's multi_logloss: 0.722441
[100]	valid_0's multi_logloss: 0.720059
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.720059


[I 2025-05-22 14:53:45,995] Trial 0 finished with value: 0.6816588876024734 and parameters: {'learning_rate': 0.2974076182133885, 'num_leaves': 49, 'max_depth': 5, 'min_child_samples': 86, 'subsample': 0.7605254586821979, 'colsample_bytree': 0.5575807832927954, 'reg_alpha': 0.012893076468883968, 'reg_lambda': 0.17026608359845577}. Best is trial 0 with value: 0.6816588876024734.


Training until validation scores don't improve for 50 rounds
[10]	valid_0's multi_logloss: 0.974519
[20]	valid_0's multi_logloss: 0.904064
[30]	valid_0's multi_logloss: 0.859508
[40]	valid_0's multi_logloss: 0.829644
[50]	valid_0's multi_logloss: 0.808655
[60]	valid_0's multi_logloss: 0.793129
[70]	valid_0's multi_logloss: 0.781096
[80]	valid_0's multi_logloss: 0.771485
[90]	valid_0's multi_logloss: 0.763596
[100]	valid_0's multi_logloss: 0.756956
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.756956


[I 2025-05-22 14:55:33,544] Trial 1 finished with value: 0.66771368198078 and parameters: {'learning_rate': 0.039771130450381234, 'num_leaves': 177, 'max_depth': 8, 'min_child_samples': 66, 'subsample': 0.8909021176741665, 'colsample_bytree': 0.6671628852333613, 'reg_alpha': 1.8484405828483973e-08, 'reg_lambda': 1.5969120042800658e-07}. Best is trial 0 with value: 0.6816588876024734.


...

Training until validation scores don't improve for 50 rounds
[10]	valid_0's multi_logloss: 0.838512
[20]	valid_0's multi_logloss: 0.776996
[30]	valid_0's multi_logloss: 0.750311
[40]	valid_0's multi_logloss: 0.735032
[50]	valid_0's multi_logloss: 0.724851
[60]	valid_0's multi_logloss: 0.717324
[70]	valid_0's multi_logloss: 0.711391
[80]	valid_0's multi_logloss: 0.706704
[90]	valid_0's multi_logloss: 0.702829
[100]	valid_0's multi_logloss: 0.699703
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.699703


[I 2025-05-22 16:39:24,245] Trial 48 finished with value: 0.6931899008883189 and parameters: {'learning_rate': 0.13453638966299902, 'num_leaves': 224, 'max_depth': 14, 'min_child_samples': 80, 'subsample': 0.6477607347218769, 'colsample_bytree': 0.7121131519039022, 'reg_alpha': 9.586184155172608e-06, 'reg_lambda': 0.02393163876880972}. Best is trial 41 with value: 0.6953756535192694.


Training until validation scores don't improve for 50 rounds
[10]	valid_0's multi_logloss: 0.906501
[20]	valid_0's multi_logloss: 0.830254
[30]	valid_0's multi_logloss: 0.791965
[40]	valid_0's multi_logloss: 0.769216
[50]	valid_0's multi_logloss: 0.754089
[60]	valid_0's multi_logloss: 0.743103
[70]	valid_0's multi_logloss: 0.734612
[80]	valid_0's multi_logloss: 0.72792
[90]	valid_0's multi_logloss: 0.722334
[100]	valid_0's multi_logloss: 0.71763
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.71763


[I 2025-05-22 16:42:07,399] Trial 49 finished with value: 0.6842080670366698 and parameters: {'learning_rate': 0.07167478158522553, 'num_leaves': 244, 'max_depth': 15, 'min_child_samples': 76, 'subsample': 0.5572078701737916, 'colsample_bytree': 0.7492229241104136, 'reg_alpha': 0.00011602969017084999, 'reg_lambda': 0.0001564256909356638}. Best is trial 41 with value: 0.6953756535192694.


✅ Best LightGBM params: {'learning_rate': 0.21062203597149756, 'num_leaves': 222, 'max_depth': 16, 'min_child_samples': 77, 'subsample': 0.6580912408833793, 'colsample_bytree': 0.7692238505371277, 'reg_alpha': 5.901408442631763e-08, 'reg_lambda': 0.0021466669288140797}
Training until validation scores don't improve for 50 rounds
[10]	valid_0's multi_logloss: 0.796554
[20]	valid_0's multi_logloss: 0.75005
[30]	valid_0's multi_logloss: 0.730373
[40]	valid_0's multi_logloss: 0.719009
[50]	valid_0's multi_logloss: 0.711462
[60]	valid_0's multi_logloss: 0.705967
[70]	valid_0's multi_logloss: 0.701927
[80]	valid_0's multi_logloss: 0.698823
[90]	valid_0's multi_logloss: 0.696379
[100]	valid_0's multi_logloss: 0.694583
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.694583
✅ LightGBM model saved to best_lightgbm.pkl


### Xgboost hyper-parameter tuning

In [ ]:
# XGBOOST OPTUNA TUNING + SAVE
# -------------------------------
def objective_xgb(trial):
    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': len(set(y)),
        'tree_method': 'gpu_hist',
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True)
    }

    model = xgb.XGBClassifier(
        **param,
        use_label_encoder=False,
        random_state=42,
    )

    model.set_params(early_stopping_rounds=50)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    preds = model.predict(X_val)
    return accuracy_score(y_val, preds)

# Run XGBoost tuning
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("✅ Best XGBoost params:", study_xgb.best_params)

# Train final XGBoost model with best params
best_params_xgb = study_xgb.best_params
best_params_xgb.update({
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'num_class': len(set(y)),
    'tree_method': 'gpu_hist',
    'use_label_encoder': False,
    'random_state': 42
})

final_xgb = xgb.XGBClassifier(**best_params_xgb)
final_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False
)

# Save final XGBoost model
joblib.dump(final_xgb, 'best_xgboost.pkl')
print("✅ XGBoost model saved to best_xgboost.pkl")

[I 2025-05-22 16:47:33,663] A new study created in memory with name: no-name-85b8f5bc-753c-40e6-93ec-d825c9560a3a
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:48:07] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:48:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:48:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/u